#  IA Generativa em um Pipeline de ETL com Python

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco
2. Seu trabalho é obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta, atualizando a lista de "news" de cada usuário.



## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [ ]:
import pandas as pd

df = pd.read_csv('dados_cli.csv', sep='\t')
user_ids = df['UserId'].tolist()
print(user_ids)

[1, 2, 3, 4]


In [ ]:
import json

# Simula a extração de dados e a estrutura esperada para a etapa de Transformação
users = []
for index, row in df.iterrows():
    users.append({
        'id': row['UserId'],
        'name': row['UserName'],
        'news': []
    })

print(json.dumps(users, indent=2))

## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [ ]:
!pip install openai

In [ ]:
openai_api_key = ''

In [ ]:
from openai import OpenAI # Importa a nova classe OpenAI

# Inicializa o cliente OpenAI com sua chave de API
client = OpenAI(api_key=openai_api_key)

def generate_ai_news(user):
  completion = client.chat.completions.create( # Atualiza a chamada para o novo método
    model="gpt-3.5-turbo", # Alterado de gpt-4 para gpt-3.5-turbo
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "dados_cli.csv",
      "description": news
  })

## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [ ]:
def update_user(user):
  response = requests.put(f"dados_cli.csv", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")